# Analysis of the Spanish football league (La Liga)

In [60]:
import os
import re
import csv
import requests
import json
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# import warnings
# warnings.filterwarnings('ignore')

In [73]:
url = 'https://cdn.saudiexchange.sa/wps/portal/saudiexchange/ourmarkets/main-market-watch/!ut/p/z1/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziTR3NDIw8LAz8LVxcnA0C3bwtPLwM_I0MzMz1w1EVGAQHmAIVBPga-xgEGbgbmOlHEaPfAAdwNCCsPwpNia-7mUGgn2Ogv5G5qYFBsBG6AixOBCvA44bg1Dz9gtzQCIPMgHQAsqCDtA!!/p0/IZ7_IPG41I82KGASC06S67RB9A0080=CZ6_5A602H80O8DDC0QFK8HJ0O2067=NJgetMainNomucMarketDetails=/?sectorParameter=&tableViewParameter=1&iswatchListSelected=NO&requestLocale=en&_=1691337352599'
response = requests.get(url)
json_data = json.loads(response.text)
companyList = json_data['data']

fields = ['sectorName', 'companySymbol', 'acrynomName', 'companyUrl']
csv_file = open('company_data.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.DictWriter(csv_file, fields)
csv_writer.writeheader()
csv_file.close()

with open('company_data.csv', "a", newline='', encoding="utf-8") as fp:
  for companyData in companyList:
    # print(companyData['sectorName'], companyData['companySymbol'], companyData['acrynomName'], companyData['companyUrl'])
    wr = csv.writer(fp, dialect='excel')
    wr.writerow([companyData['sectorName'], companyData['companySymbol'], companyData['acrynomName'], companyData['companyUrl']])

df_company = pd.read_csv('company_data.csv')
df_company

,sectorName,companySymbol,acrynomName,companyUrl
0,Energy,2030,SARCO,/wps/portal/saudiexchange/hidden/company-profi...
1,Energy,2222,SAUDI ARAMCO,/wps/portal/saudiexchange/hidden/company-profi...
2,Energy,2380,PETRO RABIGH,/wps/portal/saudiexchange/hidden/company-profi...
3,Energy,2381,ARABIAN DRILLING,/wps/portal/saudiexchange/hidden/company-profi...
4,Energy,4030,BAHRI,/wps/portal/saudiexchange/hidden/company-profi...
...,...,...,...,...
223,Real Estate Mgmt & Dev't,4300,DAR ALARKAN,/wps/portal/saudiexchange/hidden/company-profi...
224,Real Estate Mgmt & Dev't,4310,KEC,/wps/portal/saudiexchange/hidden/company-profi...
225,Real Estate Mgmt & Dev't,4320,ALANDALUS,/wps/portal/saudiexchange/hidden/company-profi...
226,Real Estate Mgmt & Dev't,4321,CENOMI CENTERS,/wps/portal/saudiexchange/hidden/company-profi...


In [74]:
fields = ['sectorName', 'symbol', 'name', 'announced', 'eligibility_date', 'distribution_date', 'distribution_way', 'dividend_per_share']
csv_file = open('data.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.DictWriter(csv_file, fields)
csv_writer.writeheader()
csv_file.close()

df_company = df_company.reset_index()

# Web data extraction

In [75]:
base_url = 'https://cdn.saudiexchange.sa'
for index, row in df_company.iterrows():

  url = base_url + row['companyUrl']
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  dividend_div = soup.find('div', class_='announcement_corporate dividends')
  nextSibling = dividend_div.find_next_sibling("div")
  with open('data.csv', "a", newline='', encoding="utf-8") as fp:
    if not nextSibling == None:
      trList = nextSibling.find('tbody').find_all('tr')
      for tr in trList:
        tdList = tr.find_all('td')
        announced = tdList[0].text
        eligibility_date = tdList[1].text
        distribution_date = tdList[2].text
        distribution_way = tdList[3].text
        dividend_per_share = tdList[4].text
        wr = csv.writer(fp, dialect='excel')
        wr.writerow([row['sectorName'], row['companySymbol'], row['acrynomName'], announced, eligibility_date, distribution_date, distribution_way, dividend_per_share])


In [76]:
df = pd.read_csv('data.csv')
df

,sectorName,symbol,name,announced,eligibility_date,distribution_date,distribution_way,dividend_per_share
0,Energy,2030,SARCO,30/03/2023,22/06/2023,19/07/2023,تحويل للحساب,1.00
1,Energy,2030,SARCO,24/03/2022,28/06/2022,14/07/2022,تحويل للحساب,0.50
2,Energy,2030,SARCO,04/03/2021,21/04/2021,10/05/2021,تحويل للحساب,0.75
3,Energy,2030,SARCO,11/03/2020,21/04/2020,10/05/2020,تحويل للحساب,1.00
4,Energy,2030,SARCO,07/03/2019,24/04/2019,08/05/2019,تحويل للحساب,0.75
...,...,...,...,...,...,...,...,...
813,Real Estate Mgmt & Dev't,4321,CENOMI CENTERS,03/07/2022,05/07/2022,26/07/2022,تحويل للحساب,0.75
814,Real Estate Mgmt & Dev't,4321,CENOMI CENTERS,24/01/2022,26/01/2022,14/02/2022,تحويل للحساب,0.75
815,Real Estate Mgmt & Dev't,4322,RETAL,01/08/2023,28/09/2023,10/10/2023,تحويل للحساب,0.16
816,Real Estate Mgmt & Dev't,4322,RETAL,01/03/2023,02/05/2023,11/05/2023,تحويل للحساب,3.00
